In [1]:
from google.colab import files
uploaded = files.upload()
#Subo los archivos a colab/ choosing the files and introducing to colab

Saving power_volume_data.parquet to power_volume_data.parquet
Saving location_data.parquet to location_data.parquet
Saving forecast_prices.csv to forecast_prices.csv


In [140]:
import pandas as pd
import numpy as np
df_location=pd.read_parquet('location_data.parquet')
df_volume=pd.read_parquet('power_volume_data.parquet')
df_forecast=pd.read_csv('forecast_prices.csv')
# Conectarse a las tablas / connect to the tables

In [141]:
from datetime import datetime as dt
df_volume['Date']=pd.to_datetime(df_volume['Date']).dt.tz_localize('Europe/Zurich')
df_forecast['Month']=pd.to_datetime(df_forecast['Month']).dt.tz_localize('Europe/Zurich')
#Converting date and month to europe/zurich

In [142]:
df_volume=df_volume.merge(df_location,on='LocationId')
#I merge it with location choosing locationid as index
df_volume=df_volume.merge(df_forecast,left_on=df_volume['Date'].dt.strftime('%Y-%m'),right_on=df_forecast['Month'].dt.strftime('%Y-%m'))
#This merge the df_volume with df_forecast choosing the time year-month choosing df_volume['date'] and df_forecast['month]

In [143]:
df_volume['TransactionValue']=df_volume.apply(lambda row: row['OffPeakPrice'] if row['Period'] >= 20 or row['Period'] < 8 else row['PeakPrice'], axis=1)
#This create Transaction value as a column and check if period is at 7-20 hours if it is, return peakprice, else offpeakprice
df_volume['PeakOffPeak']=df_volume['Period'].apply(lambda x: 'Peak' if 8 <= x < 20 else 'OffPeak')
#This create PeakOffPeak putting peak if its in period or offpeak if not

In [144]:
df_volume['OffPeakPrice']=df_volume['OffPeakPrice'].astype(float)
df_volume['PeakPrice']=df_volume['PeakPrice'].astype(float)
df_volume['Volume']=df_volume['Volume'].astype(float)
df_volume['ForecastCost'] = df_volume.apply(lambda row: (-1 if row['BuySell'] == 'Buy' else 1) * (row['Volume'] * row['OffPeakPrice'] if row['PeakOffPeak'] == 'OffPeak' else row['Volume'] * row['PeakPrice']), axis=1)
#Calculate the forecastcost taking consideration of the direction and peakoffpeak, need to convert to float

In [145]:
df_unido = df_volume.groupby(['Month', 'LocationName', 'PeakOffPeak','Book']).agg(
    TotalVolume=('Volume', 'sum'),
    TotalTradeValue=('TransactionValue', 'sum'),
    ForecastCost=('ForecastCost', 'sum'),
).reset_index()
df_unido['ForecastPnL'] = df_unido['TotalTradeValue'] - df_unido['ForecastCost']
#Making a dataframe grouped and calculating total volume totaltrade value and forecast cost


In [146]:
from google.colab import files
for book, df_book in df_unido.groupby('Book'):
    file_name = f'{book}_PnL_Report.csv'
    df_book.to_csv(file_name, index=False)
    files.download(file_name)
#I have to make this because im doing it with colab, this make a file with the name of the book +_Pnl_report.csv

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>